In [1]:
import pandas as pd
import os
import json
import sys
import time
import numpy as np
from collections import defaultdict
import os 


os.chdir(r'F:\360MoveData\Users\Administrator\Desktop')
df = pd.read_excel(r'data.xlsx',sheet_name=0)
df.head()


result = []
for i, row in df.iterrows():
    colums = row.index.values
    d= defaultdict(list)
    for colum in colums:
        try:
            d[colum] = json.loads(row[colum])
            data=json.loads(row[colum])['data']
            info=json.loads(row[colum])['info']
            label=json.loads(row[colum])['type']
            colum_dest = colum
            if label=='TXT':
                if isinstance(data, list):
                    for i in data:
                        if isinstance(i, dict):
                            d['start'].append(i.get('start'))
                            d['columnIndex'].append(i.get('columnIndex'))
                            d['end'].append(i.get('end'))
                            d['annotateType'].append(i.get('annotateType'))
                            if isinstance(i.get('info'), list):
                                for j in i.get('info'):
                                    if isinstance(j.get('val'), list):
                                        for k in j.get('val'):
                                            if isinstance(k, dict):
                                                d['data-realLabelValue'].append(k.get('realLabelValue'))
                if isinstance(info, list):
                     for g in info:
                        for k in g.get('val'):
                            if isinstance(k, dict):
                                d['info-realLabelValue'].append(k.get('realLabelValue'))
                            else:
                                d['info-val'].append(k)
                                
            elif label=='PHOTO':
                if isinstance(data, list):
                    for i in data:
                        if isinstance(i, dict):
                            d['shapeType'].append(i.get('shapeType'))
                            d['points'].append(i.get('points'))
                            if isinstance(i.get('info'), list):
                                    for j in i.get('info'):
                                        if isinstance(j.get('val'), list):
                                            for k in j.get('val'):
                                                if isinstance(k, dict):
                                                    d['data-realLabelValue'].append(k.get('realLabelValue'))
                if isinstance(info, list):
                     for g in info:
                        if isinstance(g.get('val'),list):
                            for k in g.get('val'):
                                if isinstance(k,dict):
                                    d['info-realLabelValue'].append(k.get('realLabelValue'))
                                else:
                                    d['info-realLabelValue'].append(k)
                        else:
                            if g:
                                d['info-val'].append(g.get('val'))
                            
        except:
            d[colum] = row[colum]
    result.append(d)

df = pd.DataFrame(result)   

if label=='TXT':
#     def fun(a,b,c):
#         if isinstance(a,list) and isinstance(b,list):
#             temp_col=list(zip(a,b))
#             print([temp_col][0])
#     m=df.apply(lambda x: fun(x['start'],x['end'],x['标注人员']),axis=1)
    
    
    df1=pd.DataFrame()
    for i in ['info-realLabelValue','start','columnIndex','end','annotateType','data-realLabelValue']:
        m=df[i].apply(lambda x : ",".join('%s' %i for i in x) if  isinstance(x, list)  else x)
        m=m.str.split(',',expand=True)
        m.columns = [i + str(col)  if  str(col).isdigit() else col  for col in m.columns]
        df1=pd.concat([df1,m],axis=1)

    df=df.join(df1)
    df.drop(columns=[colum_dest,'info-realLabelValue','start','columnIndex','end','annotateType','data-realLabelValue'], inplace=True)

    
if label=='PHOTO':
    df=df.explode('points')
    df1=pd.DataFrame()
    for i in ['info-realLabelValue','shapeType','data-realLabelValue']:
        m=df[i].apply(lambda x : ",".join('%s' %i for i in x) if  isinstance(x, list)  else x)
        m=m.str.split(',',expand=True)
        m.columns = [i + str(col)  if  str(col).isdigit() else col  for col in m.columns]
        df1=pd.concat([df1,m],axis=1)

    df=df.join(df1)
    df.drop(columns=[colum_dest,'info-realLabelValue','shapeType','data-realLabelValue'], inplace=True)

    
df.replace({None:np.nan},inplace=True)
df.reset_index(drop=True, inplace=True)
df=df.iloc[df.astype(str).drop_duplicates().index]
df.reset_index(drop=True, inplace=True)


# start_cols=[i for i in df.columns if i.startswith('start')]
# end_cols=[i for i in df.columns if i.startswith('end')]
# temp_col=list(zip(start_cols,end_cols))

# n=pd.DataFrame()
# for i in range(len(temp_col)):
#     m=df.apply(lambda x: df['标注人员'][x[str(temp_col[i][1])]:x[str(temp_col[i][0])]] if  x[str(temp_col[i][0])] ,axis=1)
    

df

,标注人员,points,info-val,info-realLabelValue0,info-realLabelValue1,info-realLabelValue2,info-realLabelValue3,info-realLabelValue4,info-realLabelValue5,info-realLabelValue6,info-realLabelValue7,shapeType0,shapeType1,shapeType2,data-realLabelValue0,data-realLabelValue1,data-realLabelValue2,data-realLabelValue3,data-realLabelValue4,data-realLabelValue5
0,王二,"[{'x': 67.04252577319585, 'y': 22.768041237113...",[整条文本内容],有效,4级标签1,美发护发,NaN,NaN,NaN,NaN,NaN,rect,rect,rect,真菌,哺乳动物,游戏,西方魔幻游戏,东方仙侠游戏,游戏（按题材）
1,王二,"[{'x': 183.32216494845355, 'y': 39.84407216494...",[整条文本内容],有效,4级标签1,美发护发,NaN,NaN,NaN,NaN,NaN,rect,rect,rect,真菌,哺乳动物,游戏,西方魔幻游戏,东方仙侠游戏,游戏（按题材）
2,王二,"[{'x': 78.42654639175257, 'y': 449.66881443298...",[整条文本内容],有效,4级标签1,美发护发,NaN,NaN,NaN,NaN,NaN,rect,rect,rect,真菌,哺乳动物,游戏,西方魔幻游戏,东方仙侠游戏,游戏（按题材）
3,张三,"[{'x': 124.50396825396824, 'y': 99.20634920634...",[这是针对整条数据的描述：小姐姐],有效,3级标签,4级标签1,3级标签2,4级标签4,4级标签5,4级标签7,厨具餐具,polygon,NaN,NaN,哺乳动物,宫斗游戏,东方仙侠游戏,NaN,NaN,NaN
4,杨六,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
